In [1]:
import pdfplumber
import pandas as pd
import re

In [2]:
import sys
print(sys.executable)


C:\Users\PC\miniconda3\envs\ml\python.exe


In [3]:
from pathlib import Path

base_dir = Path(r"D:\Documents\fish")

pdf_files = list(base_dir.rglob("*.pdf"))

print(f"Found {len(pdf_files)} PDF files")
for pdf in pdf_files[:10]:   # preview first 10
    print(pdf)


Found 139 PDF files
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\1.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\10.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\11.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\12.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\13.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\14.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\2.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\3.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\4.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\5.pdf


In [4]:
type(pdf_files)

list

In [5]:
def extract_pdf_data(pdf_path):
    records = []

    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]  # assuming all data on first page
        table = page.extract_table()
        text = page.extract_text()

        if not text or not table:
            return records

        # ---- Extract metadata (SAFE regex) ----
        metadata = {}
        try:
            metadata["farmer_name"] = re.search(
                r"Farmer's Name:\s*(.*?)\s*S\.No\.:", text
            ).group(1)
            metadata["area"] = re.search(
                r"Area:\s*(.*?)\s*No\.Of Samples:", text
            ).group(1)
            metadata["sample_no"] = re.search(
                r"S\.No\.\:\s*(\S+)", text
            ).group(1)
            metadata["date_time"] = re.search(
                r"Date:\s*(.*)", text
            ).group(1)
            metadata["no_of_samples"] = re.search(
                r"No\.Of Samples:\s*(\d+)", text
            ).group(1)
        except AttributeError:
            print(f"⚠️ Metadata regex failed for {pdf_path.name}")
            return records

        # ---- Extract table ----
        df = pd.DataFrame(table[1:], columns=table[0])

        df.columns = (
            df.columns
              .str.strip()
              .str.lower()
              .str.replace(" ", "_")
        )

        df = df.replace(["NIL", ""], None)
        df = df.dropna(how="all")

        # ---- Merge metadata into each row ----
        for _, row in df.iterrows():
            rec = metadata.copy()
            rec.update(row.to_dict())
            rec["file_name"] = pdf_path.name
            rec["folder_path"] = str(pdf_path.parent)
            records.append(rec)

    return records


In [6]:
from tqdm.notebook import tqdm
all_records = []

#for pdf_path in pdf_files:
for pdf_path in tqdm(pdf_files, desc="Processing PDFs"):
    #print(f"Processing: {pdf_path.relative_to(base_dir)}")
    all_records.extend(extract_pdf_data(pdf_path))

all_df = pd.DataFrame(all_records)
#all_df.head()


Processing PDFs:   0%|          | 0/139 [00:00<?, ?it/s]

In [7]:
all_df.to_csv("D:\Documents\water-tests-prediction\data\lab_data_clean.csv", index=False)


In [8]:
columns_to_keep = [
    "farmer_name",
    "area",
    "acre",
    "pond",
    "d.o.c",
    "date_time",
    "folder_path",
    "file_name",
    "do"
]
simple_df = all_df[columns_to_keep].copy()
#clean data
simple_df.loc[
    simple_df["farmer_name"].astype(str).str.lower().str.contains("venk"),
    "farmer_name"
] = "Mr. Venkata Raju"

simple_df.loc[
    simple_df["farmer_name"].astype(str).str.lower().str.contains("prasad"),
    "farmer_name"
] = "Mr. Prasad Raju"

simple_df.loc[
    simple_df["area"].astype(str).str.lower().str.contains("ai bhima"),
    "area"
] = "AI-BHIMAVARAM"

simple_df.loc[
    simple_df["area"].astype(str).str.lower().str.contains("aibhimavaram"),
    "area"
] = "AI-BHIMAVARAM"

#add pond_id column
simple_df["pond_id"] = (simple_df["farmer_name"] + "_" + simple_df["area"] + "_" + simple_df["acre"]
)

In [9]:
display(simple_df[
    simple_df["area"].str.contains("AIBHIMAVARAM", case=False, na=False)
]
       )


,farmer_name,area,acre,pond,d.o.c,date_time,folder_path,file_name,do,pond_id


In [10]:
display(simple_df[
    simple_df["farmer_name"].str.contains("prasa", case=False, na=False)
]
       )


,farmer_name,area,acre,pond,d.o.c,date_time,folder_path,file_name,do,pond_id
139,Mr. Prasad Raju,THARATAVA,1,SHP,50,20-Jan-2025 09:05,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,1.pdf,4.5,Mr. Prasad Raju_THARATAVA_1
140,Mr. Prasad Raju,THARATAVA,1,SHP,45,25-Aug-2025 08:54,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,10.pdf,5.0,Mr. Prasad Raju_THARATAVA_1
141,Mr. Prasad Raju,THARATAVA,1,SHP,52,03-Sep-2025 08:59,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,11.pdf,4.5,Mr. Prasad Raju_THARATAVA_1
142,Mr. Prasad Raju,THARATAVA,1,SHP,60,11-Sep-2025 09:10,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,12.pdf,5.0,Mr. Prasad Raju_THARATAVA_1
143,Mr. Prasad Raju,THARATAVA,1,SHP,65,20-Sep-2025 09:11,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,13.pdf,5.0,Mr. Prasad Raju_THARATAVA_1
144,Mr. Prasad Raju,THARATAVA,1,SHP,None,22-Sep-2025 09:12,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,14.pdf,5.0,Mr. Prasad Raju_THARATAVA_1
145,Mr. Prasad Raju,THARATAVA,1,E.T,None,14-Oct-2025 09:16,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,15.pdf,4.0,Mr. Prasad Raju_THARATAVA_1
146,Mr. Prasad Raju,THARATAVA,1,SHP,17,02-Nov-2025 08:33,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,16.pdf,5.0,Mr. Prasad Raju_THARATAVA_1
147,Mr. Prasad Raju,THARATAVA,1,SHP,45,01-Dec-2025 08:09,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,17.pdf,4.0,Mr. Prasad Raju_THARATAVA_1
148,Mr. Prasad Raju,THARATAVA,1,SHP,None,07-Dec-2025 09:35,D:\Documents\fish\PRASAD THARATAVA\PRASAD THAR...,18.pdf,4.5,Mr. Prasad Raju_THARATAVA_1


In [11]:
pond_counts = simple_df.groupby("pond_id").size().sort_values(ascending=False)

print(pond_counts)


pond_id
VEM AGRO TECH_APPARAO PETA_14                   28
VEM AGRO TECH_APPARAO PETA_18                   27
Mr. Prasad Raju_THARATAVA_1                     21
MR.JAGANADA RAJU GARU_KONDURU_4                 14
MR.JAGANNADHA RAJU GARU_KONDURU_4               13
Mr. Venkata Raju_AI-BHIMAVARAM_1.5              11
MR.RAVI TEJA_RAJULAPETA_7                       10
Mr. Venkata Raju_AI-BHIMAVARAM_6                 6
Mr. Venkata Raju_AI-BHIMAVARAM_2                 6
MR.GANESH GARU_KONDURU_4                         6
MR.GANESH GARU_KONDURU_1                         6
MR.GANESH GARU_KONDURU_3                         5
MR.THRINADHA MURTHY RAJU GARU_KALAVAPUDI_3       4
MR.GANESH GARU_KONDURU_2                         3
MR.THRINADHA MURTHY RAJU GARU_KALAVAPUDI_2       3
VEM AGRO TECH_APPARAO PETA_56                    3
VEM AGRO TECH_APPARAO PETA_55                    3
MR.THRINADA MURTHY RAJU GARU_KALAVAPUDI_3        2
Mr. Venkata Raju_AI-BHIMAVARAM_18                2
MR.THRINADA MURTHY RAJU

In [12]:
simple_df1 = simple_df[
    simple_df.groupby("pond_id")["pond_id"].transform("count") >= 3
].copy()


In [13]:
pond_counts = simple_df1.groupby("pond_id").size().sort_values(ascending=False)

print(pond_counts)


pond_id
VEM AGRO TECH_APPARAO PETA_14                 28
VEM AGRO TECH_APPARAO PETA_18                 27
Mr. Prasad Raju_THARATAVA_1                   21
MR.JAGANADA RAJU GARU_KONDURU_4               14
MR.JAGANNADHA RAJU GARU_KONDURU_4             13
Mr. Venkata Raju_AI-BHIMAVARAM_1.5            11
MR.RAVI TEJA_RAJULAPETA_7                     10
MR.GANESH GARU_KONDURU_1                       6
MR.GANESH GARU_KONDURU_4                       6
Mr. Venkata Raju_AI-BHIMAVARAM_6               6
Mr. Venkata Raju_AI-BHIMAVARAM_2               6
MR.GANESH GARU_KONDURU_3                       5
MR.THRINADHA MURTHY RAJU GARU_KALAVAPUDI_3     4
MR.THRINADHA MURTHY RAJU GARU_KALAVAPUDI_2     3
MR.GANESH GARU_KONDURU_2                       3
VEM AGRO TECH_APPARAO PETA_55                  3
VEM AGRO TECH_APPARAO PETA_56                  3
dtype: int64


In [14]:
print(simple_df1["pond_id"].nunique())
print(simple_df1.shape)


17
(169, 10)


In [15]:
simple_df1["date_time"] = pd.to_datetime(simple_df1["date_time"], errors="coerce")
simple_df1["do"] = pd.to_numeric(simple_df1["do"], errors="coerce")

simple_df1 = simple_df1.dropna(subset=["date_time", "do"])

In [16]:
simple_df1 = simple_df1.sort_values(["pond_id", "date_time"])

simple_df1["days_since_last_test"] = (
    simple_df1.groupby("pond_id")["date_time"]
    .diff()
    .dt.days
)

simple_df1["DO_lag1"] = (
    simple_df1.groupby("pond_id")["do"]
    .shift(1)
)

simple_df1 = simple_df1.dropna(subset=["DO_lag1", "days_since_last_test"])


In [17]:
print("Final rows:", simple_df1.shape[0])
print("Unique ponds:", simple_df1["pond_id"].nunique())
print(simple_df1.head())


Final rows: 152
Unique ponds: 17
       farmer_name     area acre pond d.o.c           date_time  \
1   MR.GANESH GARU  KONDURU    1  SHP    30 2025-02-05 09:53:00   
22  MR.GANESH GARU  KONDURU    1  E.T  None 2025-02-13 11:21:00   
20  MR.GANESH GARU  KONDURU    1  E.T     - 2025-03-10 13:35:00   
12  MR.GANESH GARU  KONDURU    1  SHP    60 2025-06-21 08:41:00   
9   MR.GANESH GARU  KONDURU    1  SHP    60 2025-08-17 09:29:00   

                                          folder_path file_name   do  \
1   D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...    10.pdf  5.0   
22  D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...     9.pdf  5.0   
20  D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...     8.pdf  5.0   
12  D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...     4.pdf  4.5   
9   D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...     2.pdf  4.5   

                     pond_id  days_since_last_test  DO_lag1  
1   MR.GANESH GARU_KONDURU_1                   6.0      4.5  
22  MR.

In [18]:
print(simple_df1[["do", "DO_lag1", "days_since_last_test"]].describe())
simple_df1 = simple_df1[simple_df1["do"] <= 10].copy()
print(simple_df1[["do", "DO_lag1", "days_since_last_test"]].describe())

               do     DO_lag1  days_since_last_test
count  152.000000  152.000000            152.000000
mean     5.069737    4.859211             27.078947
std      3.300533    0.464980             38.140821
min      3.000000    3.000000              0.000000
25%      4.500000    4.500000              7.000000
50%      5.000000    5.000000             14.000000
75%      5.000000    5.000000             25.250000
max     45.000000    5.500000            223.000000
               do     DO_lag1  days_since_last_test
count  151.000000  151.000000            151.000000
mean     4.805298    4.861589             27.119205
std      0.515983    0.465598             38.264505
min      3.000000    3.000000              0.000000
25%      4.500000    4.500000              7.000000
50%      5.000000    5.000000             14.000000
75%      5.000000    5.000000             25.500000
max      5.500000    5.500000            223.000000


In [19]:
simple_df1[simple_df1["do"] > 10]


,farmer_name,area,acre,pond,d.o.c,date_time,folder_path,file_name,do,pond_id,days_since_last_test,DO_lag1


In [20]:
split_date = simple_df1["date_time"].quantile(0.8)

train_df = simple_df1[simple_df1["date_time"] <= split_date]
test_df  = simple_df1[simple_df1["date_time"] > split_date]

print("Train rows:", train_df.shape[0])
print("Test rows:", test_df.shape[0])


Train rows: 121
Test rows: 30


In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

features = ["DO_lag1", "days_since_last_test"]

X_train = train_df[features]
y_train = train_df["do"]

X_test = test_df[features]
y_test = test_df["do"]

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))


MAE: 0.5779181177618677
R2: -2.5078549377700394


In [22]:
baseline_pred = X_test["DO_lag1"]

print("Baseline MAE:", mean_absolute_error(y_test, baseline_pred))
print("Baseline R2:", r2_score(y_test, baseline_pred))


Baseline MAE: 0.5333333333333333
Baseline R2: -2.092783505154639


In [25]:
import joblib

joblib.dump(model, "D:/Documents/water-tests-prediction/models/do_model.pkl")


['D:/Documents/water-tests-prediction/models/do_model.pkl']